# <center>CHINOOK RECORD STORE</center>

## <center>Answering Business Questions to aid Strategic Business Decision Making </center>

### 1.1     Intoduction and Database Schema

Chinook Record Store is a multinational record store carrying out the business of purchasing tracks and albums from record labels to sell to the final consumer. The business have decided to have a concrete analysis on its activities over the past 3 years (2017-2020) to what improvement needs to be made in respect to maximising revenue.

We were provided with a Database that can be downloaded under the repository for this project alongside the sql script on my Github. The database contains 11 tables and the database schema can be seen below.

![Database_Schema](https://s3.amazonaws.com/dq-content/191/chinook-schema.svg)

### 1.2 Establishing a Database Connection with Jupyter Notebook

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## <center>Working with the Datasets </center>

### 1.3 The Genres with highest sales in USA  
#### What are people listening to?

The Chinook record store has just signed a deal with a new record label, and the task is to select the first three album that will be added to the store, from a list of four. All four albums are by artist that don't have any tracks in the store right now - we have the artist names, and genre of music they produce:

|Artist Name          |Genre   |
|---------------------|--------|
|Regal                |Hip-Hop |
|Red Tone             |Punk    |
|Meteor and the Girls |Pop     |
|Slim Jim Bites       |Blues   |

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in USA, it is therefore of key interest to find out which genres sell the best in USA.

I will be writing a query to find out which genres sell the most tracks in USA in order to make a recommendation for the three artists whose albums we should purchase for the store

In [11]:
%%sql

SELECT 
    g.name genre_name,
    SUM(il.quantity) total_quantity_sold,
    (CAST(SUM(il.quantity) AS FLOAT) / 
         (SELECT COUNT(il.quantity)
          FROM invoice_line il
          JOIN invoice i ON il.invoice_id = i.invoice_id
          WHERE billing_country = 'USA')) percentage_of_total_quantity_sold
FROM genre g
    JOIN track t ON g.genre_id = t.genre_id
    JOIN invoice_line il ON t.track_id = il.track_id
    JOIN invoice i ON il.invoice_id = i.invoice_id
WHERE billing_country = 'USA'
GROUP BY genre_name
ORDER BY total_quantity_sold DESC;

Done.


genre_name,total_quantity_sold,percentage_of_total_quantity_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


From the result above, Rock genre is the highest genre sold in USA with a total of 561 units (53.37% of total) sold within the given period, followed by Alternative & Punk genre with a total of 130 units sold (12.4% of total) and Metal genre with a total of 124 units sold (11.8% of total) respectively.

In respect to required recommendation for the artists' albums to be purchased for the store, it can be seen that Punk has the highest sales out of the four genres with a 12% sale of the market share while the Blues genre and the pop genre have a 3.4% and 2% sales respectively, and Hip-Hop genre coming last with a 1.9% sale of the market share.

Therefore, i would recommend to purchase the album from the following artists:

* Red Tone (Punk)
* Slim Jim Bites (Blues)
* Meteor and the Girls (Pop)

### 1.4 The Sales Support Agent Sales Performance 
#### Who's performing better or worse?

Each customer for the Chinook record store gets assigned a sales support agent within the company when they make their first purchase. It is therefore imperative to analyze each purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others and consider the possible factors for the performance.

In [12]:
%%sql 
SELECT
    e.employee_id employee_id,
    e.first_name || " " || e.last_name employee_name,
    e.title employee_title,
    e.hire_date employment_date,
    SUM(i.total) total_sales,
    (SUM(i.total) / 
    (SELECT SUM(i.total) 
            FROM employee e
            JOIN customer c ON e.employee_id = c.support_rep_id
            JOIN invoice i ON c.customer_id = i.customer_id
            WHERE title = 'Sales Support Agent'))percentage_of_total_sales
FROM employee e
    JOIN customer c ON e.employee_id = c.support_rep_id
    JOIN invoice i ON c.customer_id = i.customer_id
WHERE title = 'Sales Support Agent'
GROUP BY employee_name
ORDER BY total_sales DESC;

Done.


employee_id,employee_name,employee_title,employment_date,total_sales,percentage_of_total_sales
3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,1731.5100000000039,0.3676686987597238
4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,1584.0000000000034,0.336346436829936
5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,1393.920000000002,0.29598486441034344


Using the hire date as a factor for judging each employee's preformance; it is clear that 'Jane Peacock' is the oldest sales support agent having being employed on April, 1st 2017 whom also happen to have the highest sales out of all sales support agent with a total sales of CAD1,731.51. The same pattern is also repeated for 'Margaret Park' - the second oldest employee; employed on May, 3rd 2017 with a total sales of CAD1,584.00. The last employee on the list is 'Steve Johnson', also the last employee to be employed. Employed on October 10th, 2017; Steve Johnson made a total sales of CAD1,393.92.

Is this sufficient to conclude Steve Johnson is performing worse? I believe the answer is NO. Despite being the last employee on the list and a difference of 5-6 months apart from other sales support agents, Steve Johnson is only 8% away from achieving the sales amount made by 'Jane Peacock' - the sales supoort agent with the highest sales.

### 1.5 Analyzing Sales and Customers by Country
#### Which countries have profit making more opportunities judging by average value of sales per customer?

The next task will be to analyze the sales data for customers from each different country. Our goal here is to find the following for each country:

* total numbers of customers
* total value of sales
* average value of sales per customer
* average order value

In [17]:
%%sql
WITH sales_table AS
    (
        SELECT
            CASE
                WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 'Others' 
                ELSE country
            END country,
            COUNT(DISTINCT(c.customer_id)) customer,
            SUM(i.total) total_sales,
            SUM(i.total) / COUNT(c.customer_id) average_order_value,
            SUM(i.total) / COUNT(DISTINCT(c.customer_id)) average_value_of_sales_per_customer,
            CASE
                WHEN COUNT(DISTINCT(c.customer_id)) = 1 THEN 1
                ELSE 0
            END sort
            FROM customer c
            JOIN invoice i ON c.customer_id = i.customer_id
            GROUP BY country
    )
    
SELECT
    country,
    CASE
        WHEN country = 'Others' THEN SUM(customer)
        ELSE customer
    END customer,
    CASE
        WHEN country = 'Others' THEN SUM(total_sales)
        ELSE total_sales
    END total_sales,
    CASE
        WHEN country = 'Others' THEN (SUM(average_order_value) / COUNT(average_order_value))
        ELSE average_order_value
    END average_order_value,
    CASE
        WHEN country = 'Others' THEN (SUM(average_value_of_sales_per_customer) / COUNT(average_value_of_sales_per_customer))
        ELSE average_value_of_sales_per_customer
    END average_value_of_sales_per_customer
FROM sales_table
GROUP BY 1
ORDER BY sort ASC, total_sales DESC

Done.


country,customer,total_sales,average_order_value,average_value_of_sales_per_customer
USA,13,1040.4899999999998,7.942671755725189,80.0376923076923
Canada,8,535.5900000000001,7.047236842105265,66.94875000000002
Brazil,5,427.68000000000006,7.0111475409836075,85.53600000000002
France,5,389.0699999999999,7.781399999999998,77.81399999999998
Germany,4,334.62,8.161463414634147,83.655
Czech Republic,2,273.24000000000007,9.108000000000002,136.62000000000003
United Kingdom,3,245.52,8.768571428571429,81.84
Portugal,2,185.13000000000002,6.383793103448276,92.56500000000001
India,2,183.14999999999998,8.72142857142857,91.57499999999999
Others,15,1094.9399999999998,7.445071062271063,72.996


Despite having low number of customers, the following countries appeared to have the highest number of average value of sales per customer:

* Czech Republic
* India
* United Kingdom

This implies that there are more profit making opportunites in these countries. However, it is important to mention that the amount of data used for this analysis is relatively low. Chinook should be cautious about embarking on a huge marketing campaigns in these countries. I would recommend smaller marketing campaigns are carried out first in order to collect and analyze data about new customers and see if this trend will continue to hold with new customers.

### 1.6 Individual Tracks vs The Whole Album 
#### Can we maximise profit by purchasing more of Individual tracks or by purchasing the whole album?

In [10]:
%%sql
WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice) percent
FROM
    (
      SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases made up one-fifth of the total revenue earned by Chinook record store over the year. I will therefore advise against purchasing only individual tracks on albums from record labels. This is ideal as purchasing only individual tracks can cause Chinook to lose 18.6% of its revenue.

### 1.7 The Listener's Choice
#### Which artist is used in most playlist? Do we buy more tracks/albums made by the artist?

In [20]:
%%sql
SELECT
    ar.artist_id artist_id, 
    ar.name artist_name,
    COUNT (DISTINCT(p.playlist_id)) number_of_playlist,
    COUNT (pt.track_id) number_of_tracks
FROM playlist p
    JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
    JOIN track t ON pt.track_id = t.track_id
    JOIN album a ON t.album_id = a.album_id
    JOIN artist ar ON a.artist_id = ar.artist_id
GROUP BY artist_name
ORDER BY number_of_playlist DESC;

Done.


artist_id,artist_name,number_of_playlist,number_of_tracks
226,Eugene Ormandy,7,15
214,Academy of St. Martin in the Fields & Sir Neville Marriner,6,10
248,Berliner Philharmoniker & Herbert Von Karajan,6,13
208,English Concert & Trevor Pinnock,6,9
247,The King's Singers,6,10
257,"Academy of St. Martin in the Fields, Sir Neville Marriner & Thurston Dart",5,5
260,Adrian Leaper & Doreen de Feis,5,5
206,Alberto Turco & Nova Schola Gregoriana,5,5
243,Antal Doráti & London Symphony Orchestra,5,5
224,Barry Wordsworth & BBC Concert Orchestra,5,5


'Eugene Omandy' appeared to be the artist featured in most playlists featuring in 7 playlists followed by the 'Academy of St. Martin in the Fields & Sir Neville Marriner', 'Berliner Philharmoniker & Herbert Von Karajan', 'English Concert & Trevor Pinnock' and 'The King's Singers' appearing on 6 playlists each. 

Given the versality of these artistes, i would recommend Chinook record store purchase more of their tracks as they stand higher chances of being sold rapidly.

### 1.8 Reviewing Sales Record of each Track
#### Are there tracks that are yet to be sold? If yes, what genres are they? Should we buy more of this genres or ignore them completely in order to reduce cost?

In [21]:
%%sql
WITH full_track_table AS
    (
        SELECT 
            DISTINCT(t.track_id) track_id,
            t.name track_name,
            il.quantity quantity
            FROM track t
            LEFT JOIN invoice_line il ON t.track_id = il.track_id
    )
    
SELECT
    CASE 
        WHEN quantity IS NOT NULL THEN 'Purchased Tracks'
        ELSE 'Unpurchased Tracks'
    END purchased_history,
    COUNT(track_id) number_of_tracks
FROM full_track_table
GROUP BY purchased_history;

Done.


purchased_history,number_of_tracks
Purchased Tracks,1806
Unpurchased Tracks,1697


This showed that 1,697 tracks are yet to be sold to any customers. I will drill down further to look at these figures on a genre level to establish if a track genre is a reason for the track not being purchased

#### 1.8.1 List of tracks sold and their respective Genres

In [22]:
%%sql
WITH full_track_table AS
    (
        SELECT 
            DISTINCT(t.track_id) track_id,
            t.name track_name,
            g.name genre_name,
            il.quantity quantity
            FROM track t
            LEFT JOIN invoice_line il ON t.track_id = il.track_id
            JOIN genre g ON t.genre_id = g.genre_id
    )
    
SELECT
    genre_name,
    CASE 
        WHEN quantity IS NOT NULL THEN 'Purchased Tracks'
        ELSE 'Unpurchased Tracks'
    END purchased_history,
    COUNT(track_id) number_of_tracks
FROM full_track_table
GROUP BY genre_name, purchased_history
HAVING purchased_history = 'Purchased Tracks'
ORDER BY number_of_tracks DESC;

Done.


genre_name,purchased_history,number_of_tracks
Rock,Purchased Tracks,915
Metal,Purchased Tracks,238
Alternative & Punk,Purchased Tracks,176
Latin,Purchased Tracks,119
Jazz,Purchased Tracks,61
Blues,Purchased Tracks,56
R&B/Soul,Purchased Tracks,55
Alternative,Purchased Tracks,34
Electronica/Dance,Purchased Tracks,29
Pop,Purchased Tracks,25


#### 1.8.2 List of tracks yet to purchased and their respective Genres

In [23]:
%%sql
WITH full_track_table AS
    (
        SELECT 
            DISTINCT(t.track_id) track_id,
            t.name track_name,
            g.name genre_name,
            il.quantity quantity
            FROM track t
            LEFT JOIN invoice_line il ON t.track_id = il.track_id
            JOIN genre g ON t.genre_id = g.genre_id
    )
    
SELECT
    genre_name,
    CASE 
        WHEN quantity IS NOT NULL THEN 'Purchased Tracks'
        ELSE 'Unpurchased Tracks'
    END purchased_history,
    COUNT(track_id) number_of_tracks
FROM full_track_table
GROUP BY genre_name, purchased_history
HAVING purchased_history = 'Unpurchased Tracks'
ORDER BY number_of_tracks DESC;

Done.


genre_name,purchased_history,number_of_tracks
Latin,Unpurchased Tracks,460
Rock,Unpurchased Tracks,382
Alternative & Punk,Unpurchased Tracks,156
Metal,Unpurchased Tracks,136
TV Shows,Unpurchased Tracks,91
Jazz,Unpurchased Tracks,69
Drama,Unpurchased Tracks,63
Classical,Unpurchased Tracks,58
Soundtrack,Unpurchased Tracks,38
Reggae,Unpurchased Tracks,36


The result clearly shows that there are 7 genres with zero sales so far and they include the followinng:

* Bossa Nova
* Comedy
* Opera
* Rock and Roll
* Sci Fi & Fantsay
* Science Fiction
* World

I would recommend that Chinook record store currently put on hold any additional order for these genres, increase the marketing and exposure for each of them, and see if this trend will continue. If yes, Chinook record store is therefore advised to put an  end to purchases of tracks on each of the genres. 